<a href="https://colab.research.google.com/github/rahiakela/genai-research-and-practice/blob/main/campusx-langchain/02_chians_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

**Reference**

https://python.langchain.com/docs/integrations/chat/google_generative_ai/

https://python.langchain.com/docs/integrations/text_embedding/google_generative_ai/

In [1]:
%%capture

%pip install -qU langchain-google-genai
%pip install langchain langchain-community grandalf

In [13]:
import os
import json

from google.colab import userdata

from langchain_google_genai import ChatGoogleGenerativeAI

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser, JsonOutputParser

from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.schema.runnable import RunnableParallel, RunnableBranch, RunnableLambda

from pydantic import BaseModel, Field
from typing import Literal

import matplotlib.pyplot as plt
from IPython.display import Image, display
%matplotlib inline

In [3]:
model = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=userdata.get('GOOGLE_API_KEY')
)

## Step #1: Basic Chain

In [ ]:
template = PromptTemplate(
    template='Generate 5 interesting facts about {topic}',
    input_variables=['topic']
)

In [ ]:
parser = StrOutputParser()

chain = template | model | parser

result = chain.invoke({"topic":"cricket"})

In [ ]:
print(result)

Okay, here are 5 interesting facts about cricket that you might not know:

1.  **The Longest Cricket Match Lasted 12 Days:** A timeless Test match between England and South Africa in Durban in 1939 went on for 12 days (with a rest day in between). It was eventually declared a draw because the English team had to catch their boat home!

2.  **Cricket Balls Used to be Stuffed with Feathers:**  Early cricket balls, dating back to the 18th century, were literally stuffed with feathers and then covered in leather. This made them unpredictable and likely quite painful to be hit by!

3.  **The "Cow Corner" is Named After Actual Cows:** The fielding position known as "Cow Corner" got its name because it was often the area where agricultural workers would graze their cows during village cricket matches.  A shot hit in that direction was considered a risky, agricultural-style slog.

4.  **Cricket is the Second Most Popular Sport in the World:**  While it might not be as globally ubiquitous as fo

In [ ]:
chain.get_graph().print_ascii()

      +-------------+      
      | PromptInput |      
      +-------------+      
             *             
             *             
             *             
    +----------------+     
    | PromptTemplate |     
    +----------------+     
             *             
             *             
             *             
+------------------------+ 
| ChatGoogleGenerativeAI | 
+------------------------+ 
             *             
             *             
             *             
    +-----------------+    
    | StrOutputParser |    
    +-----------------+    
             *             
             *             
             *             
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  


##Step #2: Sequential Chain

In [ ]:
template1 = PromptTemplate(
    template='Generate a detailed report on {topic}',
    input_variables=['topic']
)

template2 = PromptTemplate(
    template='Generate a 5 pointer summary from the following text \n {text}',
    input_variables=['text']
)

In [ ]:
parser = StrOutputParser()

chain = template1 | model | parser | template2 | model | parser

result = chain.invoke({"topic":"Unemployment in India"})

In [ ]:
print(result)

Here's a 5-point summary of the provided text on unemployment in India:

*   **Unemployment in India is a multifaceted problem characterized by informality, underemployment, and regional disparities, masked by relatively low official unemployment rates.** The true extent of the issue lies in the lack of quality employment opportunities.

*   **Key drivers of unemployment include rapid population growth, slow economic growth, a dominant informal sector, skills gaps, and rigid labor laws.** Social factors like the caste system and gender inequality also contribute.

*   **India experiences various types of unemployment, including open, disguised, seasonal, structural, and underemployment.** Underemployment is a particularly significant issue.

*   **Unemployment has significant economic and social consequences, including reduced economic output, increased poverty, higher crime rates, social unrest, and negative health impacts.**

*   **The Indian government has implemented various initia

In [ ]:
chain.get_graph().print_ascii()

      +-------------+      
      | PromptInput |      
      +-------------+      
             *             
             *             
             *             
    +----------------+     
    | PromptTemplate |     
    +----------------+     
             *             
             *             
             *             
+------------------------+ 
| ChatGoogleGenerativeAI | 
+------------------------+ 
             *             
             *             
             *             
    +-----------------+    
    | StrOutputParser |    
    +-----------------+    
             *             
             *             
             *             
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  
             *             
             *             
             *             
    +----------------+     
    | PromptTemplate |     
    +----------------+     
             *             
             *             
             *      

##Step #3: Parallel Chain

In [4]:
prompt1 = PromptTemplate(
    template='Generate short and simple notes from the following text \n {text}',
    input_variables=['text']
)

prompt2 = PromptTemplate(
    template='Generate 5 short question answers from the following text \n {text}',
    input_variables=['text']
)

prompt3 = PromptTemplate(
    template='Merge the provided notes and quiz into a single document \n notes -> {notes} and quiz -> {quiz}',
    input_variables=['notes', 'quiz']
)

In [5]:
parser = StrOutputParser()

In [7]:
parallel_chain = RunnableParallel({
    "notes":prompt1 | model | parser,
    "quiz":prompt2 | model | parser
})

merge_chain = prompt3 | model | parser
chain = parallel_chain | merge_chain

In [15]:
chain.get_graph().print_ascii()

                    +---------------------------+                      
                    | Parallel<notes,quiz>Input |                      
                    +---------------------------+                      
                       ***                   ***                       
                   ****                         ****                   
                 **                                 **                 
    +----------------+                          +----------------+     
    | PromptTemplate |                          | PromptTemplate |     
    +----------------+                          +----------------+     
             *                                           *             
             *                                           *             
             *                                           *             
+------------------------+                  +------------------------+ 
| ChatGoogleGenerativeAI |                  | ChatGoogleGenerati

In [8]:
text = """
Support vector machines (SVMs) are a set of supervised learning methods used for classification, regression and outliers detection.

The advantages of support vector machines are:

Effective in high dimensional spaces.

Still effective in cases where number of dimensions is greater than the number of samples.

Uses a subset of training points in the decision function (called support vectors), so it is also memory efficient.

Versatile: different Kernel functions can be specified for the decision function. Common kernels are provided, but it is also possible to specify custom kernels.

The disadvantages of support vector machines include:

If the number of features is much greater than the number of samples, avoid over-fitting in choosing Kernel functions and regularization term is crucial.

SVMs do not directly provide probability estimates, these are calculated using an expensive five-fold cross-validation (see Scores and probabilities, below).

The support vector machines in scikit-learn support both dense (numpy.ndarray and convertible to that by numpy.asarray) and sparse (any scipy.sparse) sample vectors as input. However, to use an SVM to make predictions for sparse data, it must have been fit on such data. For optimal performance, use C-ordered numpy.ndarray (dense) or scipy.sparse.csr_matrix (sparse) with dtype=float64.
"""

In [9]:
result = chain.invoke({"text": text})

In [10]:
print(result)

Okay, here's the merged document containing the SVM notes and the quiz questions with answers:

**Support Vector Machines (SVMs)**

**Definition:** Supervised learning methods for classification, regression, and outlier detection.

**Advantages:**

*   Effective in high dimensional spaces.
*   Works well when dimensions > samples.
*   Memory efficient (uses support vectors).
*   Versatile (various kernel functions, including custom ones).

**Disadvantages:**

*   Overfitting risk if features >> samples (kernel/regularization tuning crucial).
*   Probability estimates are computationally expensive.

**Implementation Notes (scikit-learn):**

*   Supports dense (NumPy arrays) and sparse (SciPy sparse matrices) data.
*   Fit model on sparse data to predict sparse data.
*   Optimal performance: C-ordered NumPy arrays (dense) or CSR sparse matrices (sparse) with `dtype=float64`.

**Quiz: Support Vector Machines**

1.  **Question:** What are SVMs used for?
    **Answer:** Classification, regr

##Step #4: Conditional Chain

In [14]:

class Feedback(BaseModel):
    sentiment: Literal["positive", "negetive"] = Field(description="Give the sentiment of the feedback")

pydantic_parser = PydanticOutputParser(pydantic_object=Feedback)
str_parser = StrOutputParser()

In [16]:
prompt1 = PromptTemplate(
    template="Classify the sentiment of the following feedback text into postive or negative \n {feedback} \n {format_instruction}",
    input_variables=['feedback'],
    partial_variables={'format_instruction':pydantic_parser.get_format_instructions()}
)

classifier_chain = prompt1 | model | pydantic_parser

In [18]:
prompt2 = PromptTemplate(
    template='Write an appropriate response to this positive feedback \n {feedback}',
    input_variables=['feedback']
)

prompt3 = PromptTemplate(
    template='Write an appropriate response to this negative feedback \n {feedback}',
    input_variables=['feedback']
)

branch_chain = RunnableBranch(
    (lambda x: x.sentiment == 'positive', prompt2 | model | str_parser),
    (lambda x: x.sentiment == 'negetive', prompt3 | model | str_parser),
    RunnableLambda(lambda x: "could not find sentiment")
)

chain = classifier_chain | branch_chain

In [19]:
chain.get_graph().print_ascii()

      +-------------+      
      | PromptInput |      
      +-------------+      
             *             
             *             
             *             
    +----------------+     
    | PromptTemplate |     
    +----------------+     
             *             
             *             
             *             
+------------------------+ 
| ChatGoogleGenerativeAI | 
+------------------------+ 
             *             
             *             
             *             
 +----------------------+  
 | PydanticOutputParser |  
 +----------------------+  
             *             
             *             
             *             
        +--------+         
        | Branch |         
        +--------+         
             *             
             *             
             *             
     +--------------+      
     | BranchOutput |      
     +--------------+      


In [20]:
print(chain.invoke({'feedback': 'This is a beautiful phone'}))

Okay, to respond to positive feedback, I need a little more context!  To give you the *best* response, tell me:

*   **What was the feedback about?** (e.g., a product, a service, a presentation, a piece of writing, a performance, etc.)
*   **Who is the feedback from?** (e.g., a customer, a boss, a colleague, a friend, the general public)
*   **What was the specific positive comment?** (Even a short summary helps!)

However, here are a few general responses you can adapt, depending on the situation:

**General Responses:**

*   "Thank you so much for your positive feedback! I really appreciate you taking the time to share your thoughts."
*   "I'm so glad to hear you had a positive experience. We really value your feedback."
*   "Thank you! It's great to know that you were happy with [the product/service/etc.]."
*   "I'm delighted to hear that! We strive to [mention what you strive for, e.g., provide excellent service, create high-quality products, etc.]."

**More Specific Responses (Exa

In [21]:
print(chain.invoke({'feedback': 'This is a very bad phone'}))

Okay, to give you the best possible response, I need a little more context.  Please tell me:

1.  **What is the negative feedback about?** (e.g., a product, a service, a website, a piece of writing, a performance, etc.)
2.  **What is the specific feedback?** (Ideally, provide the exact text of the feedback. If you can't, summarize the main points.)
3.  **What is your role in relation to the feedback?** (e.g., are you the business owner, a customer service representative, the creator of the product, etc.)

However, here are a few general response templates you can adapt, depending on the situation:

**Template 1:  Acknowledging and Promising Action (Good for most situations)**

> "Thank you for your feedback. We appreciate you taking the time to share your experience. We're sorry to hear that you [mention the specific issue]. We're taking your comments seriously and will be [explain what action you'll take, e.g., investigating the issue, reviewing our processes, etc.]. We value your bus